In [1]:
import os
from types import SimpleNamespace

import dotenv
import openai
import pymongo
import pycmarkgfm
import numpy as np
from IPython import display
from ipywidgets import HTML
from bson import ObjectId

In [2]:
def connect_to_db(collection):
    dotenv.load_dotenv()
    mongodb_uri = os.getenv("MONGODB_URI")
    DBclient = pymongo.MongoClient(mongodb_uri)
    db = DBclient["ML2-Abschlussprojekt"]
    return db[collection]

In [3]:
def get_semester(id):
    collection = connect_to_db("Semester_Data")
    return collection.find_one({"_id":ObjectId(id)})

In [4]:
def get_student(id):
    collection = connect_to_db("Base_Data")
    return collection.find_one({"_id":ObjectId(id)})

In [5]:
def get_api_key():
    dotenv.load_dotenv()
    os.getenv("OPENAI_APIKEY")
    return os.getenv("OPENAI_APIKEY")

In [6]:
def _openai(messages, chain=True) -> openai.types.chat.chat_completion.Choice:
    """
    Request is sent to OpenAI
    Return the response in a ChatCompletion-Object
    """
    openai.api_key = get_api_key()
    model_result = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2,
        top_p=1,
    )
    response = model_result.choices[0]
    if chain:
        chain = messages + [response.message]
        return response.message.content, chain
    return response.message.content

In [7]:
def anonymise(student, text):
    if student["gender"].lower()=="männlich":
        aliasname = "John"
    else:
        aliasname = "Jane"
    alias_text = text.replace(student["firstname"], aliasname)
    return alias_text

In [8]:
def deanonymise(student, text):
    if student["gender"].lower()=="männlich":
        aliasname = "John"
    else:
        aliasname = "Jane"
    alias_text = text.replace(aliasname, student["firstname"])
    return alias_text

In [9]:
def format_prompt_content(semester, student):
    allgemeines_lernen = {
        "AktivTeilnehmen": 'aktiv am Unterricht teilnehmen',
        "LeistungZeigen": 'im Unterricht Leistung zeigen',
        "AufmerksamSein": 'während dem Unterricht Aufmerksam sein und den Instruktionen der Lehrperson folgen',
        "SchulinhalteMerken": 'sich schulische Inhalte merken',
        "SchulinhalteAbrufen": 'schulische Inhalte bei Bedarf abrufen',
    }
    assessment = semester["final_assessment"]["allgemeines_lernen"]
    text = "\n".join(
        f" - {student['firstname']} kann {v['assessment']} {allgemeines_lernen[k]} {v['notes']}"
        for k, v in assessment.items()
    )
    return anonymise(student, text)

In [10]:
def prompt_for_textgeneration(semester, student):
    system_content = f"""
    Du wirst eine Evaluation eines Schülers oder einer Schülerin bekommen.
    Die Evaluation wird in kurzen einfachen Sätzen sein.
    Dein Task ist es, diese Sätze zu einem Fliesstext zusammenzubringen.
    Der Fliesstext soll professionell sein.
    Der Fliesstext soll nur Informationen enthalten, die von der Evaluation mitgegeben werden.
    Der Fliesstext soll die Schweizer Rechtschreibung nutzen, das heisst, Scharf-S wird durch Doppel-S ersetzt.
    """
    first_prompt = f"""
    Bitte mache aus der folgenden Einschätzung einen professionellen Fliesstext:
    {format_prompt_content(semester, student)}
    """
    return [
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": """
    John Doe kann gut aktiv am Unterricht teilnehmen.
    John Doe kann gut im schulischen Umfeld Leistung zeigen.
    John Doe kann nicht immer während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
    John Doe kann teilweise sich schulische Inhalte merken.
    John Doe kann noch nicht schulische Inhalte bei Bedarf abrufen.
            """
        },
        {
            "role": "assistant",
            "content": """
    John Doe gelingt es gut, aktiv am Unterricht teilzunehmen.
    Ausserdem zeigt er meist gute Leistungen im schulischen Umfeld.
    Er kann jedoch während des Unterrichts nicht immer Aufmerksam sein und den Instruktionen der Lehrperson folgen.
    Ausserdem hat John Doe teilweise Mühe, sich schulische Inhalte zu merken.
    Auch gelingt es ihm noch nicht, die gemerkten Inhalte bei Bedarf abzurufen.
            """
        },
        {
            "role": "user",
            "content": """
    Jane Doe kann teilweise aktiv am Unterricht teilnehmen.
    Jane Doe kann nicht immer im schulischen Umfeld Leistung zeigen.
    Jane Doe kann gut während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
    Jane Doe kann gut sich schulische Inhalte merken.
    Jane Doe kann teilweise schulische Inhalte bei Bedarf abrufen.
            """
        },
        {
            "role": "assistant",
            "content": """
    Jane Doe kann nur teilweise aktiv am Unterricht teilnehmen.
    Nicht immer gelingt es ihr im schulischen Umfeld Leistungen zu zeigen.
    Sie kann jedoch gut während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.
    Ausserdem merkt sie sich schulische Inhalte gut, kann diese jedoch nur teilweise bei Bedarf abrufen.
            """
        },
        {
            "role": "user",
            "content": first_prompt
        },
    ]

In [11]:
def get_first_text(data):
    messages = prompt_for_textgeneration(data["semester"], data["student"])
    print(messages) # to check, if anonymise() worked
    answer, chain = _openai(messages)
    return deanonymise(data["student"], answer), chain

In [13]:
def prompt_for_follow_up(chain, prompt):
    chain.append({
        "role":"user",
        "content":prompt
    })

In [14]:
def get_other_text(data):
    prompt_for_follow_up(data["chain"], anonymise(data["student"], data["prompt"]))
    print("Checkpoint D: ", data["chain"]) # to check, if anonymise() worked
    answer, chain = _openai(messages)
    return deanonymise(data["student"], answer), chain

In [15]:
semester = get_semester("666159b1129b60e0dd53c2e1")
student = get_student(semester["student_id"])

TypeError: 'NoneType' object is not subscriptable

In [21]:
data1 = {
    "semester":semester,
    "student":student,
}

In [27]:
answer, chain = get_first_text(data1)

[{'role': 'system', 'content': '\n    Du wirst eine Evaluation eines Schülers oder einer Schülerin bekommen.\n    Die Evaluation wird in kurzen einfachen Sätzen sein.\n    Dein Task ist es, diese Sätze zu einem Fliesstext zusammenzubringen.\n    Der Fliesstext soll professionell sein.\n    Der Fliesstext soll nur Informationen enthalten, die von der Evaluation mitgegeben werden.\n    Der Fliesstext soll die Schweizer Rechtschreibung nutzen, das heisst, Scharf-S wird durch Doppel-S ersetzt.\n    '}, {'role': 'user', 'content': '\n    John Doe kann gut aktiv am Unterricht teilnehmen.\n    John Doe kann gut im schulischen Umfeld Leistung zeigen.\n    John Doe kann nicht immer während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.\n    John Doe kann teilweise sich schulische Inhalte merken.\n    John Doe kann noch nicht schulische Inhalte bei Bedarf abrufen.\n            '}, {'role': 'assistant', 'content': '\n    John Doe gelingt es gut, aktiv am Unterricht t

In [33]:
prompt = "Könntest Du zusätzlich zu Physik und Chemie noch Geografie und Geschichte schreiben?"

In [34]:
data2 = {
    "semester":semester,
    "student":student,
    "chain":chain,
    "prompt":prompt,
}

In [37]:
chain.append({"role":"user", "content":"TestTest"})

In [30]:
new_answer, new_chain = get_other_text(data2)

Checkpoint A:  [{'role': 'system', 'content': '\n    Du wirst eine Evaluation eines Schülers oder einer Schülerin bekommen.\n    Die Evaluation wird in kurzen einfachen Sätzen sein.\n    Dein Task ist es, diese Sätze zu einem Fliesstext zusammenzubringen.\n    Der Fliesstext soll professionell sein.\n    Der Fliesstext soll nur Informationen enthalten, die von der Evaluation mitgegeben werden.\n    Der Fliesstext soll die Schweizer Rechtschreibung nutzen, das heisst, Scharf-S wird durch Doppel-S ersetzt.\n    '}, {'role': 'user', 'content': '\n    John Doe kann gut aktiv am Unterricht teilnehmen.\n    John Doe kann gut im schulischen Umfeld Leistung zeigen.\n    John Doe kann nicht immer während des Unterrichts Aufmerksam sein und den Instruktionen der Lehrperson folgen.\n    John Doe kann teilweise sich schulische Inhalte merken.\n    John Doe kann noch nicht schulische Inhalte bei Bedarf abrufen.\n            '}, {'role': 'assistant', 'content': '\n    John Doe gelingt es gut, aktiv 

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages': expected an array of objects, but got null instead.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'invalid_type'}}